In [ ]:
# -*- coding: utf-8 -*-

import os
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
import warnings

# Suppress specific Scikit-learn warnings about memory leaks if needed,
# although updating scikit-learn is usually the best solution.
# warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')
# Or more specific warning suppression if you know the exact message/cause
warnings.filterwarnings(
    "ignore",
    message="KMeans is known to have a memory leak on Windows with MKL",
    category=UserWarning
)


def quantize_image_kmeans(image_path, k, output_path=None):
    """
    Reduces the number of colors in an image using K-Means clustering.

    Args:
        image_path (str): Path to the input image file
                          (supports formats like TIFF, JPEG, BMP, PNG).
        k (int): The desired number of colors (clusters) in the output image.
        output_path (str, optional): Path to save the processed image.
                                     If None, it defaults to saving in the same
                                     directory with '_quantized_k{k}' suffix.

    Returns:
        bool: True if successful, False otherwise.
    """
    try:
        # 1. Load the image
        print(f"Loading image: {image_path}")
        img = Image.open(image_path)
        img_format = img.format # Store original format for saving later if needed

        # 2. Ensure image is in RGB format for color clustering
        #    (Handles grayscale, RGBA, palette-based images etc.)
        img_rgb = img.convert('RGB')

        # 3. Prepare data for K-Means
        #    Convert image data to a NumPy array
        data = np.array(img_rgb)
        original_shape = data.shape # Store original shape (height, width, channels)

        #    Reshape the data into a 2D array where each row is a pixel's RGB value
        #    (total_pixels, 3)
        pixels = data.reshape(-1, 3)
        print(f"Image dimensions: {original_shape[1]}x{original_shape[0]} pixels.")
        print(f"Total pixels: {pixels.shape[0]}")
        print(f"Applying K-Means clustering to find {k} dominant colors...")

        # 4. Apply K-Means clustering
        #    n_init='auto' (or 10) runs the algorithm multiple times with different
        #    centroid seeds and picks the best result to avoid poor local optima.
        #    random_state ensures reproducibility.
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto', verbose=0)
        kmeans.fit(pixels)

        # 5. Get the dominant colors (cluster centers) and labels
        #    Cluster centers are the average color (RGB) for each cluster.
        #    Convert them to uint8 for image representation.
        dominant_colors = kmeans.cluster_centers_.astype(np.uint8)
        print(f"Found {len(dominant_colors)} dominant colors (centroids).")

        #    Labels indicate which cluster (dominant color) each pixel belongs to.
        labels = kmeans.labels_

        # 6. Create the new image data
        #    Replace each original pixel with its corresponding dominant color.
        #    This uses efficient NumPy indexing.
        new_pixels = dominant_colors[labels]

        #    Reshape the data back to the original image dimensions
        new_data = new_pixels.reshape(original_shape)

        # 7. Create the new image object
        new_image = Image.fromarray(new_data, 'RGB')
        print("New image data generated.")

        # 8. Save the new image
        if output_path is None:
            base, ext = os.path.splitext(image_path)
            # Use PNG by default for lossless saving, or try to keep original format
            # Using PNG is often safer for quantized images to avoid compression artifacts.
            output_ext = ".png" # Or use ext if you want to keep original format (might add artifacts for JPG)
            output_path = f"{base}_quantized_k{k}{output_ext}"

        print(f"Saving quantized image to: {output_path}")
        # Ensure the output directory exists
        output_dir = os.path.dirname(output_path)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)

        new_image.save(output_path) # Pillow determines format from extension
        print("Image saved successfully.")
        return True

    except FileNotFoundError:
        print(f"Error: Input image file not found at '{image_path}'")
        return False
    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc() # Print detailed traceback for debugging
        return False

# --- Main Execution ---
if __name__ == "__main__":
    print("--- Image Color Quantization using K-Means ---")

    while True:
        input_image_path = input("Enter the path to the image file: ").strip()
        if os.path.exists(input_image_path):
            break
        else:
            print(f"Error: File not found at '{input_image_path}'. Please try again.")

    while True:
        try:
            num_colors_str = input("Enter the desired number of colors (e.g., 3, 5, 8): ").strip()
            num_colors = int(num_colors_str)
            if num_colors > 0:
                break
            else:
                print("Error: Number of colors must be a positive integer.")
        except ValueError:
            print("Error: Invalid input. Please enter an integer number.")

    # Optional: Ask for output path or use default
    # output_image_path = input("Enter the desired output path (leave blank for default): ").strip()
    # if not output_image_path:
    #    output_image_path = None # Use default naming convention

    # Run the quantization
    success = quantize_image_kmeans(input_image_path, num_colors) # Pass output_image_path if defined

    if success:
        print("Processing finished.")
    else:
        print("Processing failed.")